In [40]:
import requests
import json
import pandas as pd

# List of years
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023]

# Initialize an empty list to store the extracted data
extracted_data = []
party_data = []

# Loop through each year
for year in years:
    # URL of the JSON data
    url = f'https://zpravy.udhpsh.cz/export/vfz{year}-index.json'

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data
        data = response.json()

        # Extract relevant data
        for party in data['parties']:
            for file in party['files']:
                extracted_data.append({
                    'year': year,
                    'key': data['election']['key'],
                    'ic': party['ic'],
                    'subject': file['subject'],
                    'url': file['url']
                })
                party_data.append({
                    'ic': party['ic'],
                    'shortName': party['shortName'],
                    'longName': party['longName']

                })
                
    else:
        print(f"Failed to retrieve data for year {year}: {response.status_code}")

# Create DataFrame
df = pd.DataFrame(extracted_data)
df_party = pd.DataFrame(party_data).drop_duplicates(subset=['ic'])
df_party.to_csv('D_party.csv', index=False)
# Print the DataFrame
print(len(df))

13040


In [ ]:
df_cprijmy = df[df['subject'] == 'cprijmy']
df_cvydaje = df[df['subject'] == 'cvydaje']
df_zamest = df[df['subject'] == 'zamest']
df_polinst = df[df['subject'] == 'polinst']
df_podil = df[df['subject'] == 'podil']
df_penizefo = df[df['subject'] == 'penizefo']
df_bupfo = df[df['subject'] == 'bupfo']
df_penizepo = df[df['subject'] == 'penizepo']
df_buppo = df[df['subject'] == 'buppo']
df_dluhy = df[df['subject'] == 'dluhy']
df_dedictvi = df[df['subject'] == 'dedictvi']
df_clenove = df[df['subject'] == 'clenove']


In [ ]:
subjects = [
df_cprijmy,
df_cvydaje,
df_zamest,
df_podil,
df_penizefo,
df_penizepo,
# df_polinst,
# df_bupfo,
# df_buppo,
# df_dluhy,
# df_dedictvi,
# df_clenove,
]

In [ ]:
def fetch_json(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        return response.json()  # Return parsed JSON
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL {url}: {e}")
        return None  # Return None in case of error

# Fetch JSON for each URL and store it in a new column
# df_head['json_response'] = df_head['url'].apply(fetch_json)
# df_head.to_csv('financovani.csv', index=False)

In [ ]:
for subject in subjects:
    subject['json_response'] = subject['url'].apply(fetch_json)
    subject.to_csv(f'{subject.iloc[0]["subject"]}.csv', index=False, encoding='utf-8')
    print(f'{subject.iloc[0]["subject"]} done')
    
    # Step 1: Expand the JSON into a tabular format
    exploded_df = subject.explode('json_response')

    # Step 2: Normalize the JSON data
    expanded_df = pd.json_normalize(exploded_df['json_response'])

    # Add meaningful context from the original DataFrame
    result_df = exploded_df[['year',  'ic']].reset_index(drop=True)
    result_df = pd.concat([result_df, expanded_df], axis=1)
    result_df.to_csv(f'/Users/zackrawr/Developer/Python_WS/public_service/CURATED/{subject.iloc[0]["subject"]}.csv', index=False, encoding='utf-8')
    print(f'{subject.iloc[0]["subject"]} done in curated')


In [43]:
zamestnanci = pd.read_csv('/Users/zackrawr/Developer/Python_WS/public_service/CURATED/zamest.csv')

{'Právník - přepočteno na FTE', 'Tiskový mluvčí', 'Krajský manažer', 'Administrativní činnosti', 'archivář', 'Asistent generálního sekretáře', 'vedení účetnictví', 'asistent - průměrný přepočtený stav 2019', 'Tisková mluvčí', 'Administrativní práce - zaokrouhlené přepočtené úvazky 2019', 'Mediální služby', 'provozní agenda hnutí', 'Dohoda o provedení práce - administrativní práce', 'DPP - účetní', 'Vedoucí zaměstnanci', 'DPP - IT služby', 'analytik', 'administrativní podpora', 'Expertizy a analýzy', 'Hlavní pokladník', 'Krajský tajemník a organizační pracovník', 'PR specialita', 'Mediace a koordinace', 'DPČ administrativa 1-5/2019', 'BOZP expert', 'tiskový mluvčí', 'pomocník v kampani', 'Tiskový mluvčí, koordinátor tiskového úseku', 'specialista pro regiony - průměrný přepočtený stav 2018', 'Organizační práce v ústředí', 'politický pracovník/pracovnice', 'technicko-hospodářský pracovník', 'Natáčení filmových spotů', 'krajský manažer - průměrný přepočtený stav 2017', 'hospodářský pracov